## Clustering

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import nyu_dataset
import pix2pixHD_model
from torch.utils.data import DataLoader 
from sklearn.cluster import KMeans
from collections import defaultdict

In [2]:
_cuda = 2

In [3]:
encoder = pix2pixHD_model.FeatureEncoder(3, 3, instance_norm=True).cuda(_cuda)
encoder.load_state_dict(torch.load('./pix2pixHD_newsegm/models/encoder_170_0'))

In [5]:
segmentation_dataset_validation = nyu_dataset.SegmentationDataset(path_to_datafolder='./datasets/nyu/val/', 
    transforms=nyu_dataset.SegmentationTransform(train_trainsforms=False, resize=False), 
    use_instance_segmentation=True)
data_loader_val = DataLoader(segmentation_dataset_validation, batch_size=1, shuffle=False, num_workers=1)

In [68]:
dictionary = {}

count = 0
for image, segm, inst, bord in data_loader_val:
    image = Variable(image.cuda(_cuda), requires_grad=False, volatile=True)
    segm = Variable(segm.cuda(_cuda), requires_grad=False, volatile=True)
    inst = inst.cuda(_cuda)
    count += 1
    print(count, end='')
    if count > 100:
        break 
      
    encoded = encoder.forward(image, inst)

    array_segm = segm.data.cpu().numpy()
    array_encoded = encoded.data.cpu().numpy()
    classes = range(array_segm.shape[1])

    for k in classes:   
        idx = np.argwhere(array_segm[0,k,...]==1)
        for x, y in idx:
            feature_array = tuple(array_encoded[0, :, x, y].tolist())
            dictionary.setdefault(k,set()).add(feature_array)

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101

In [93]:
cluster_centers = defaultdict(list)
for k, features in dictionary.items():
    if len(features)<10:
        continue
    kmeans = KMeans(n_clusters = 10).fit([list(feat) for feat in features])
    k_centers = kmeans.cluster_centers_
    cluster_centers[k].append(k_centers)

## Number of instance and number of classes

In [96]:
i =0 
for image, segm, inst, bord in data_loader_val:
    image = Variable(image.cuda(_cuda), requires_grad=False, volatile=True)
    segm = Variable(segm.cuda(_cuda), requires_grad=False, volatile=True)
    inst = inst.cuda(_cuda)
    i += 1
    if i == 25:
        break

In [97]:
classes_map = np.argmax(segm[0].data.cpu().numpy(), axis=0)

instance = inst.cpu().numpy()

print(len(np.unique(classes_map)))
print(len(np.unique(instance)))

16
8
